## Titanic Snake
  
Esse caderno tem o objetivo de construir uma aplicação em python para o titanic do Kaggle.  
Para completar essa task é preciso que o usuário faça uma predição de se um passageiro do _Titanic_ sobreviveu com base em um dataset disponibilizado no arquivo train.csv e deve ser dado um resultado para esses fatores com o arquivo test.csv.  
O Formato de submissão do arquivo deve ser o de um csv com duas columns com os cabeçalhos _PassengerId_ e _suvived_. Esse resultado será avaliado pela Kaggle e dará o resultado do quão boa é sua solução.

**Obs**: A recomendação da Kaggle é o uso de binary classification

In [1]:
## Import libs
import pandas
import numpy as np
import keras

Using TensorFlow backend.
/Users/caiogomes/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
## Load datasets
train_dataset = pandas.read_csv("train.csv")
test_dataset = pandas.read_csv("test.csv")

## Remove as tables desnecessárias. OBS: O PassengerId não importa nos dados de treinamento mas no output sim
train_dataset = train_dataset.drop(columns=["Name", "Ticket", "Cabin"]) # TODO: Reavaliar a importância de SibSp, parch
test_dataset = test_dataset.drop(columns=["Name", "Ticket", "Cabin"])

## Remove every single NaN and puts 0 because it will ignore the data input in the inference model
train_dataset["Embarked"] = train_dataset["Embarked"].fillna("C") # trata o caso das strings da classe de embarque
test_dataset["Embarked"] = test_dataset["Embarked"].fillna("C") # trata o caso das strings da classe de embarque
train_dataset = train_dataset.fillna(0)
test_dataset = test_dataset.fillna(0)

In [3]:
## Define como categorizar uma column
def cathegoryList(lst):
    st = sorted(set(lst))
    cathegories = dict((b,a) for a,b in enumerate(st))
    return cathegories

def cathegorize(lst):
    cathegories = cathegoryList(lst)
    lst = [cathegories[a] for a in lst]
    return lst

animals = cathegorize(["cat", "dog", "dog", "perl", "omastar", "cat", "dog"])
print(animals)

[0, 1, 1, 3, 2, 0, 1]


In [4]:
## Apply a number encoding for every string cathegory of the set
sex_cathegory = cathegoryList(test_dataset["Sex"].tolist())
test_dataset["Sex"] = test_dataset.apply(lambda x: sex_cathegory[x["Sex"]], axis=1)

embarked_cathegory = cathegoryList(test_dataset["Embarked"].tolist())
test_dataset["Embarked"] = test_dataset.apply(lambda x: embarked_cathegory[x["Embarked"]], axis=1)

sex_cathegory = cathegoryList(train_dataset["Sex"].tolist())
train_dataset["Sex"] = train_dataset.apply(lambda x: sex_cathegory[x["Sex"]], axis=1)

embarked_cathegory = cathegoryList(train_dataset["Embarked"].tolist())
train_dataset["Embarked"] = train_dataset.apply(lambda x: embarked_cathegory[x["Embarked"]], axis=1)

print(train_dataset[:20])

    PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0             1         0       3    1  22.0      1      0   7.2500         2
1             2         1       1    0  38.0      1      0  71.2833         0
2             3         1       3    0  26.0      0      0   7.9250         2
3             4         1       1    0  35.0      1      0  53.1000         2
4             5         0       3    1  35.0      0      0   8.0500         2
5             6         0       3    1   0.0      0      0   8.4583         1
6             7         0       1    1  54.0      0      0  51.8625         2
7             8         0       3    1   2.0      3      1  21.0750         2
8             9         1       3    0  27.0      0      2  11.1333         2
9            10         1       2    0  14.0      1      0  30.0708         0
10           11         1       3    0   4.0      1      1  16.7000         2
11           12         1       1    0  58.0      0      0  26.5

In [5]:
## Normaliza os dados para inserir em uma rede neural posteriormente
for col in train_dataset:
    if col != "PassengerId" and col != "Survived" and col != "Sex":
        train_dataset[col] = (train_dataset[col] - train_dataset[col].min())/(train_dataset[col].max() - train_dataset[col].min())

print(train_dataset[:20])

    PassengerId  Survived  Pclass  Sex     Age  SibSp     Parch      Fare  \
0             1         0     1.0    1  0.2750  0.125  0.000000  0.014151   
1             2         1     0.0    0  0.4750  0.125  0.000000  0.139136   
2             3         1     1.0    0  0.3250  0.000  0.000000  0.015469   
3             4         1     0.0    0  0.4375  0.125  0.000000  0.103644   
4             5         0     1.0    1  0.4375  0.000  0.000000  0.015713   
5             6         0     1.0    1  0.0000  0.000  0.000000  0.016510   
6             7         0     0.0    1  0.6750  0.000  0.000000  0.101229   
7             8         0     1.0    1  0.0250  0.375  0.166667  0.041136   
8             9         1     1.0    0  0.3375  0.000  0.333333  0.021731   
9            10         1     0.5    0  0.1750  0.125  0.000000  0.058694   
10           11         1     1.0    0  0.0500  0.125  0.166667  0.032596   
11           12         1     0.0    0  0.7250  0.000  0.000000  0.051822   

In [6]:
## Splita os dados de treinamento em train e test
train = train_dataset.sample(frac=0.8)
test = train_dataset.drop(index=train.index)
print(train.shape)

(713, 9)


In [37]:
## Monta o modelo
from keras.models import Sequential
from keras.layers import Dense, Dropout

n_classes = 1

model = Sequential()
model.add(Dense(10, input_shape=(7,))) # Qualquer número de samples e 7 features do tensor
# model.add(Dense(10, input_shape=(2,))) # Qualquer número de samples e 7 features do tensor
for i in range(0, 15):
    model.add(Dense(150, activation="tanh"))
    model.add(Dropout(0.4))
model.add(Dense(n_classes, activation="sigmoid"))

# For a binary classification problem
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 10)                80        
_________________________________________________________________
dense_65 (Dense)             (None, 150)               1650      
_________________________________________________________________
dropout_46 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_66 (Dense)             (None, 150)               22650     
_________________________________________________________________
dropout_47 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_67 (Dense)             (None, 150)               22650     
_________________________________________________________________
dropout_48 (Dropout)         (None, 150)               0         
__________

In [38]:
## One hot encode the data labels
to_ignore = ["PassengerId", "Survived", "SibSp", "Parch", "Fare", "Embarked", "Pclass"]
# model.fit(train.drop(["PassengerId", "Survived"], axis=1), train["Survived"], epochs=1000, batch_size=100)
model.fit(train.drop(["PassengerId", "Survived"], axis=1), train["Survived"], epochs=500)
# model.fit(train.drop(to_ignore, axis=1), train["Survived"], epochs=1000, batch_size=100)

Epoch 1/1000
713/713 [==============================] - 2s 3ms/step - loss: 0.7648 - acc: 0.5021
Epoch 2/1000
713/713 [==============================] - 0s 196us/step - loss: 0.7465 - acc: 0.5680
Epoch 3/1000
713/713 [==============================] - 0s 197us/step - loss: 0.7134 - acc: 0.5610
Epoch 4/1000
713/713 [==============================] - 0s 192us/step - loss: 0.6896 - acc: 0.6073
Epoch 5/1000
713/713 [==============================] - 0s 204us/step - loss: 0.6201 - acc: 0.6844
Epoch 6/1000
713/713 [==============================] - 0s 191us/step - loss: 0.5739 - acc: 0.7363
Epoch 7/1000
713/713 [==============================] - 0s 195us/step - loss: 0.5785 - acc: 0.7714
Epoch 8/1000
713/713 [==============================] - 0s 189us/step - loss: 0.5476 - acc: 0.7756
Epoch 9/1000
713/713 [==============================] - 0s 192us/step - loss: 0.5519 - acc: 0.7644
Epoch 10/1000
713/713 [==============================] - 0s 191us/step - loss: 0.5926 - acc: 0.7574
Epoch 11/10

713/713 [==============================] - 0s 212us/step - loss: 0.4674 - acc: 0.7910
Epoch 84/1000
713/713 [==============================] - 0s 207us/step - loss: 0.4625 - acc: 0.8107
Epoch 85/1000
713/713 [==============================] - 0s 201us/step - loss: 0.4821 - acc: 0.7938
Epoch 86/1000
713/713 [==============================] - 0s 200us/step - loss: 0.5246 - acc: 0.7980
Epoch 87/1000
713/713 [==============================] - 0s 210us/step - loss: 0.5056 - acc: 0.8022
Epoch 88/1000
713/713 [==============================] - 0s 205us/step - loss: 0.4992 - acc: 0.7994
Epoch 89/1000
713/713 [==============================] - 0s 201us/step - loss: 0.5115 - acc: 0.7938
Epoch 90/1000
713/713 [==============================] - 0s 188us/step - loss: 0.4916 - acc: 0.8079
Epoch 91/1000
713/713 [==============================] - 0s 198us/step - loss: 0.5006 - acc: 0.8036
Epoch 92/1000
713/713 [==============================] - 0s 207us/step - loss: 0.4852 - acc: 0.8022
Epoch 93/1000


713/713 [==============================] - 0s 182us/step - loss: 0.4993 - acc: 0.8022
Epoch 165/1000
713/713 [==============================] - 0s 180us/step - loss: 0.4939 - acc: 0.7840
Epoch 166/1000
713/713 [==============================] - 0s 178us/step - loss: 0.4794 - acc: 0.7840
Epoch 167/1000
713/713 [==============================] - 0s 180us/step - loss: 0.4780 - acc: 0.8022
Epoch 168/1000
713/713 [==============================] - 0s 184us/step - loss: 0.4856 - acc: 0.7798
Epoch 169/1000
713/713 [==============================] - 0s 180us/step - loss: 0.4903 - acc: 0.7672
Epoch 170/1000
713/713 [==============================] - 0s 175us/step - loss: 0.4813 - acc: 0.7882
Epoch 171/1000
713/713 [==============================] - 0s 181us/step - loss: 0.5151 - acc: 0.7616
Epoch 172/1000
713/713 [==============================] - 0s 176us/step - loss: 0.5275 - acc: 0.7840
Epoch 173/1000
713/713 [==============================] - 0s 179us/step - loss: 0.5260 - acc: 0.7756
Epoch

Epoch 245/1000
713/713 [==============================] - 0s 205us/step - loss: 0.4791 - acc: 0.8135
Epoch 246/1000
713/713 [==============================] - 0s 199us/step - loss: 0.5161 - acc: 0.7868
Epoch 247/1000
713/713 [==============================] - 0s 183us/step - loss: 0.4977 - acc: 0.8022
Epoch 248/1000
713/713 [==============================] - 0s 190us/step - loss: 0.4982 - acc: 0.8008
Epoch 249/1000
713/713 [==============================] - 0s 196us/step - loss: 0.4929 - acc: 0.7980
Epoch 250/1000
713/713 [==============================] - 0s 209us/step - loss: 0.4874 - acc: 0.8050
Epoch 251/1000
713/713 [==============================] - 0s 192us/step - loss: 0.4756 - acc: 0.8121
Epoch 252/1000
713/713 [==============================] - 0s 193us/step - loss: 0.4853 - acc: 0.8065
Epoch 253/1000
713/713 [==============================] - 0s 194us/step - loss: 0.4791 - acc: 0.8079
Epoch 254/1000
713/713 [==============================] - 0s 191us/step - loss: 0.4711 - ac

713/713 [==============================] - 0s 177us/step - loss: 0.5052 - acc: 0.8036
Epoch 327/1000
713/713 [==============================] - 0s 176us/step - loss: 0.4961 - acc: 0.8050
Epoch 328/1000
713/713 [==============================] - 0s 177us/step - loss: 0.5031 - acc: 0.8036
Epoch 329/1000
713/713 [==============================] - 0s 179us/step - loss: 0.4961 - acc: 0.8050
Epoch 330/1000
713/713 [==============================] - 0s 178us/step - loss: 0.4945 - acc: 0.8065
Epoch 331/1000
713/713 [==============================] - 0s 182us/step - loss: 0.5004 - acc: 0.8036
Epoch 332/1000
713/713 [==============================] - 0s 176us/step - loss: 0.4982 - acc: 0.8036
Epoch 333/1000
713/713 [==============================] - 0s 179us/step - loss: 0.5131 - acc: 0.8036
Epoch 334/1000
713/713 [==============================] - 0s 176us/step - loss: 0.4972 - acc: 0.8036
Epoch 335/1000
713/713 [==============================] - 0s 177us/step - loss: 0.4978 - acc: 0.8036
Epoch

...

713/713 [==============================] - 0s 178us/step - loss: 0.5976 - acc: 0.7139
Epoch 651/1000
713/713 [==============================] - 0s 184us/step - loss: 0.5966 - acc: 0.7167
Epoch 652/1000
713/713 [==============================] - 0s 197us/step - loss: 0.5954 - acc: 0.7153
Epoch 653/1000
713/713 [==============================] - 0s 200us/step - loss: 0.5884 - acc: 0.7181
Epoch 654/1000
713/713 [==============================] - 0s 183us/step - loss: 0.5913 - acc: 0.7195
Epoch 655/1000
713/713 [==============================] - 0s 182us/step - loss: 0.5791 - acc: 0.7251
Epoch 656/1000
713/713 [==============================] - 0s 184us/step - loss: 0.5804 - acc: 0.7237
Epoch 657/1000
713/713 [==============================] - 0s 214us/step - loss: 0.5818 - acc: 0.7265
Epoch 658/1000
713/713 [==============================] - 0s 225us/step - loss: 0.5839 - acc: 0.7237
Epoch 659/1000
713/713 [==============================] - 0s 190us/step - loss: 0.5679 - acc: 0.7251
Epoch

713/713 [==============================] - 0s 205us/step - loss: 0.5623 - acc: 0.7307
Epoch 732/1000
713/713 [==============================] - 0s 204us/step - loss: 0.5569 - acc: 0.7321
Epoch 733/1000
713/713 [==============================] - 0s 212us/step - loss: 0.5572 - acc: 0.7349
Epoch 734/1000
713/713 [==============================] - 0s 201us/step - loss: 0.5593 - acc: 0.7321
Epoch 735/1000
713/713 [==============================] - 0s 213us/step - loss: 0.5626 - acc: 0.7321
Epoch 736/1000
713/713 [==============================] - 0s 211us/step - loss: 0.5665 - acc: 0.7321
Epoch 737/1000
713/713 [==============================] - 0s 202us/step - loss: 0.5607 - acc: 0.7307
Epoch 738/1000
713/713 [==============================] - 0s 205us/step - loss: 0.5547 - acc: 0.7321
Epoch 739/1000
713/713 [==============================] - 0s 201us/step - loss: 0.5579 - acc: 0.7307
Epoch 740/1000
713/713 [==============================] - 0s 213us/step - loss: 0.5552 - acc: 0.7335
Epoch

713/713 [==============================] - 0s 178us/step - loss: 0.5693 - acc: 0.7293
Epoch 813/1000
713/713 [==============================] - 0s 181us/step - loss: 0.5600 - acc: 0.7307
Epoch 814/1000
713/713 [==============================] - 0s 182us/step - loss: 0.5625 - acc: 0.7265
Epoch 815/1000
713/713 [==============================] - 0s 183us/step - loss: 0.5634 - acc: 0.7307
Epoch 816/1000
713/713 [==============================] - 0s 182us/step - loss: 0.5666 - acc: 0.7279
Epoch 817/1000
713/713 [==============================] - 0s 193us/step - loss: 0.5656 - acc: 0.7279
Epoch 818/1000
713/713 [==============================] - 0s 180us/step - loss: 0.5556 - acc: 0.7279
Epoch 819/1000
713/713 [==============================] - 0s 180us/step - loss: 0.5683 - acc: 0.7293
Epoch 820/1000
713/713 [==============================] - 0s 182us/step - loss: 0.5665 - acc: 0.7293
Epoch 821/1000
713/713 [==============================] - 0s 182us/step - loss: 0.5640 - acc: 0.7279
Epoch

713/713 [==============================] - 0s 204us/step - loss: 0.5682 - acc: 0.7265
Epoch 894/1000
713/713 [==============================] - 0s 190us/step - loss: 0.5691 - acc: 0.7265
Epoch 895/1000
713/713 [==============================] - 0s 207us/step - loss: 0.5758 - acc: 0.7279
Epoch 896/1000
713/713 [==============================] - 0s 210us/step - loss: 0.5618 - acc: 0.7265
Epoch 897/1000
713/713 [==============================] - 0s 208us/step - loss: 0.5657 - acc: 0.7251
Epoch 898/1000
713/713 [==============================] - 0s 207us/step - loss: 0.5589 - acc: 0.7265
Epoch 899/1000
713/713 [==============================] - 0s 205us/step - loss: 0.5703 - acc: 0.7251
Epoch 900/1000
713/713 [==============================] - 0s 207us/step - loss: 0.5580 - acc: 0.7265
Epoch 901/1000
713/713 [==============================] - 0s 202us/step - loss: 0.5686 - acc: 0.7265
Epoch 902/1000
713/713 [==============================] - 0s 190us/step - loss: 0.5734 - acc: 0.7237
Epoch

713/713 [==============================] - 0s 204us/step - loss: 0.5100 - acc: 0.8065
Epoch 975/1000
713/713 [==============================] - 0s 211us/step - loss: 0.5512 - acc: 0.7994
Epoch 976/1000
713/713 [==============================] - 0s 203us/step - loss: 0.5320 - acc: 0.8050
Epoch 977/1000
713/713 [==============================] - 0s 208us/step - loss: 0.5261 - acc: 0.8022
Epoch 978/1000
713/713 [==============================] - 0s 199us/step - loss: 0.5026 - acc: 0.8050
Epoch 979/1000
713/713 [==============================] - 0s 195us/step - loss: 0.5127 - acc: 0.8022
Epoch 980/1000
713/713 [==============================] - 0s 197us/step - loss: 0.5172 - acc: 0.8008
Epoch 981/1000
713/713 [==============================] - 0s 211us/step - loss: 0.5116 - acc: 0.8022
Epoch 982/1000
713/713 [==============================] - 0s 208us/step - loss: 0.5045 - acc: 0.8022
Epoch 983/1000
713/713 [==============================] - 0s 208us/step - loss: 0.5099 - acc: 0.7980
Epoch

In [39]:
# model.evaluate(train.drop(["PassengerId", "Survived"], axis=1), train["Survived"], batch_size=100)
model.evaluate(train.drop(["PassengerId", "Survived"], axis=1), train["Survived"])
# model.evaluate(train.drop(to_ignore, axis=1), train["Survived"], batch_size=100)

713/713 [==============================] - 0s 676us/step


[0.49726331210989105, 0.788218793828892]

In [10]:
val = model.predict(test_dataset.drop(["PassengerId"], axis=1))
val = val.reshape((val.shape[0]))
val = [int(a > 0.5) for a in val]

output = test_dataset["PassengerId"].copy()
output = output.to_frame()
print(output.shape, len(val))
print(output)
output = output.assign(Survived=val)

filename = "titanic_survivors.csv"
output.to_csv(filename, sep=',', encoding='utf-8', header=True, index=False)

(418, 1) 418
     PassengerId
0            892
1            893
2            894
3            895
4            896
5            897
6            898
7            899
8            900
9            901
10           902
11           903
12           904
13           905
14           906
15           907
16           908
17           909
18           910
19           911
20           912
21           913
22           914
23           915
24           916
25           917
26           918
27           919
28           920
29           921
..           ...
388         1280
389         1281
390         1282
391         1283
392         1284
393         1285
394         1286
395         1287
396         1288
397         1289
398         1290
399         1291
400         1292
401         1293
402         1294
403         1295
404         1296
405         1297
406         1298
407         1299
408         1300
409         1301
410         1302
411         1303
412         1304
413         1305
4